In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

/Users/wy/anaconda/lib/python2.7/site-packages/cffi/model.py:525: UserWarning: 'point_conversion_form_t' has no values explicitly defined; guessing that it is equivalent to 'unsigned int'
  % self._get_c_name())


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
"""
Simple tester for the vgg19_trainable
"""

import tensorflow as tf

import vgg19_trainable as vgg19
import utils

img1 = utils.load_image("./test_data/tiger.jpeg")
img1_true_result = [1 if i == 292 else 0 for i in range(1000)]  # 1-hot result for tiger

batch1 = img1.reshape((1, 224, 224, 3))

/Users/wy/anaconda/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [3]:
print batch1.shape
bat = mnist.train.next_batch(10)

(1, 224, 224, 3)


In [4]:
print bat[0].shape
print bat[1].shape
b0 = bat[0].reshape((-1,28,28,1)).repeat(8,axis=1).repeat(8,axis=2).repeat(3,axis=3)
print b0.shape
print bat[1]

(10, 784)
(10, 10)
(10, 224, 224, 3)
[[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]]


In [5]:
import numpy as np

In [6]:
#with tf.device('/cpu:0'):
sess = tf.Session()

batch_size = 5

images = tf.placeholder(tf.float32, [None, 224, 224, 3])
true_out = tf.placeholder(tf.float32, [None, 10])
train_mode = tf.placeholder(tf.bool)

#vgg = vgg19.Vgg19('./vgg19.npy')
vgg = vgg19.Vgg19()
vgg.build(images, train_mode)

# print number of variables used: 143667240 variables, i.e. ideal size = 548MB
print(vgg.get_var_count())

sess.run(tf.global_variables_initializer())
#sess.run(tf.initialize_all_variables())


bat = mnist.train.next_batch(batch_size)
batch = bat[0].reshape((-1,28,28,1)).repeat(8,axis=1).repeat(8,axis=2).repeat(3,axis=3)
batchout = bat[1]

# test classification
prob = sess.run(vgg.prob, feed_dict={images: batch, train_mode: False})
print "finish"
print np.argmax(prob, axis = 1), np.argmax(batchout, axis=1)

139611210
finish
[1 1 1 1 1] [1 6 3 0 7]


In [7]:
# simple 1-step training
#sess.run(tf.global_variables_initializer())
print vgg.prob
print true_out
temp_1 = vgg.prob - true_out
temp_2 = temp_1 ** 2
cost = tf.reduce_sum(temp_2)
cost = tf.reduce_mean(vgg.prob)
print temp_1
print temp_2
print cost, cost.get_shape()
optim = tf.train.GradientDescentOptimizer(0.0001)
#print optim
train = optim.minimize(cost)
#print train

correct_prediction = tf.equal(tf.argmax(vgg.prob, 1), tf.argmax(true_out, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

for i in range(100):
    bat = mnist.train.next_batch(batch_size)
    batch = bat[0].reshape((-1,28,28,1)).repeat(8,axis=1).repeat(8,axis=2).repeat(3,axis=3)
    batchout = bat[1]
    sess.run(train, feed_dict={images: batch, true_out: batchout, train_mode: True})
    print 'hello', i
    if i % 10 == 0:
        val = mnist.validation.next_batch(10)
        val_batch = val[0].reshape((-1,28,28,1)).repeat(8,axis=1).repeat(8,axis=2).repeat(3,axis=3)
        val_batchout = val[1]
        print(sess.run(accuracy, feed_dict={images: val_batch, true_out: val_batchout, train_mode: False}))
        

# test classification again, should have a higher probability about tiger
prob = sess.run(vgg.prob, feed_dict={images: batch1, train_mode: False})
utils.print_prob(prob[0], './synset.txt')

# test save
#vgg.save_npy(sess, './test-save.npy')

Tensor("prob:0", shape=(?, 10), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 10), dtype=float32)
Tensor("sub_3:0", shape=(?, 10), dtype=float32)
Tensor("pow:0", shape=(?, 10), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32) ()
hello 0
0.0
hello 1
hello 2
hello 3
hello 4
hello 5
hello 6
hello 7
hello 8
hello 9
hello 10
0.1
hello 11
hello 12
hello 13
hello 14
hello 15
hello 16
hello 17
hello 18
hello 19
hello 20
0.1
hello 21
hello 22
hello 23
hello 24
hello 25
hello 26
hello 27
hello 28
hello 29
hello 30
0.1
hello 31
hello 32
hello 33
hello 34
hello 35
hello 36
hello 37
hello 38
hello 39
hello 40
0.1
hello 41
hello 42
hello 43
hello 44
hello 45
hello 46
hello 47
hello 48
hello 49
hello 50
0.0
hello 51
hello 52
hello 53
hello 54
hello 55
hello 56
hello 57
hello 58
hello 59
hello 60
0.1
hello 61
hello 62
hello 63
hello 64
hello 65
hello 66
hello 67
hello 68
hello 69
hello 70
0.2
hello 71
hello 72
hello 73
hello 74
hello 75
hello 76
hello 77
hello 78
hello 79
hello 80
0.1
hel

KeyboardInterrupt: 